In [106]:
#IMPORTANDO OS MÓDULOS
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [107]:
#LENDO O DATASET DE JULHO

dfJul = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("C:/Users/Diego/NASA_access_log_Jul95")

In [108]:
#LENDO O DATASET DE AGOSTO

dfAug = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("C:/Users/Diego/NASA_access_log_Aug95")

In [109]:
#JUNTANDO OS DATASETS

df = dfJul.union(dfAug)

df.show(7)

+--------------------+---+---+--------------------+------+--------------------+---+----+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6| _c7|
+--------------------+---+---+--------------------+------+--------------------+---+----+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200|6245|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|3985|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4085|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|   0|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4179|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|   0|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|   0|
+--------------------+---+---+--------------------+------+--------------------+---+----+
only showing top 7 ro

In [110]:
#SEPARAÇÃO E TRATAMENTO DOS DADOS

split_col = f.split(df['_c3'], ':')
df = df.withColumn('_c3', split_col.getItem(0))

df = df.withColumn('_c3', df['_c3'].substr(2, 11))

df = df.selectExpr('_c0 as host', '_c3 as date', '_c6 as code', '_c7 as size')

df.show(7)

+--------------------+-----------+----+----+
|                host|       date|code|size|
+--------------------+-----------+----+----+
|        199.72.81.55|01/Jul/1995| 200|6245|
|unicomp6.unicomp.net|01/Jul/1995| 200|3985|
|      199.120.110.21|01/Jul/1995| 200|4085|
|  burger.letters.com|01/Jul/1995| 304|   0|
|      199.120.110.21|01/Jul/1995| 200|4179|
|  burger.letters.com|01/Jul/1995| 304|   0|
|  burger.letters.com|01/Jul/1995| 200|   0|
+--------------------+-----------+----+----+
only showing top 7 rows



In [111]:
#CONTAGEM DOS HOST'S UNICOS = 137979

df.select('host').distinct().count()

137979

In [112]:
#TOTAL DE ERROS 404 = 20871

df.where(df['code'] == "404").count()

20871

In [113]:
#5 URL QUE MAIS CAUSARAM ERROS
#|hoohoo.ncsa.uiuc.edu
#|piweba3y.prodigy.com
#|jbiagioni.npt.nuw...
#|piweba1y.prodigy.com
#|www-d4.proxy.aol.com

df.where(df['code'] == "404").groupBy("host").count().orderBy("count", ascending=False).show(5)
df.where(df['code'] == "404").groupBy("date").count().orderBy("date", ascending=True).show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc.edu|  251|
|piweba3y.prodigy.com|  156|
|jbiagioni.npt.nuw...|  132|
|piweba1y.prodigy.com|  114|
|www-d4.proxy.aol.com|   91|
+--------------------+-----+
only showing top 5 rows

+-----------+-----+
|       date|count|
+-----------+-----+
|01/Aug/1995|  243|
|01/Jul/1995|  316|
|02/Jul/1995|  291|
|03/Aug/1995|  303|
|03/Jul/1995|  470|
|04/Aug/1995|  346|
|04/Jul/1995|  359|
|05/Aug/1995|  236|
|05/Jul/1995|  497|
|06/Aug/1995|  373|
|06/Jul/1995|  640|
|07/Aug/1995|  537|
|07/Jul/1995|  569|
|08/Aug/1995|  390|
|08/Jul/1995|  302|
|09/Aug/1995|  279|
|09/Jul/1995|  348|
|10/Aug/1995|  306|
|10/Jul/1995|  398|
|11/Aug/1995|  263|
+-----------+-----+
only showing top 20 rows



In [114]:
#TOTAL DE BYTES RETORNADOS =65524319796

total = df.groupBy().agg(f.sum('size'))
total.withColumn('sum(size)', total['sum(size)'].cast(DecimalType(18, 0))).show()

dfJul.count()

dfAug.count()

+-----------+
|  sum(size)|
+-----------+
|65524319796|
+-----------+



1569898